# ラボ 2: LangGraph コンポーネント

## 環境の設定

In [ ]:
from dotenv import load_dotenv
import json
import os
import re
import sys
import warnings

import boto3
from botocore.config import Config

warnings.filterwarnings("ignore")
import logging

# import local modules
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Load environment variables from .env file or Secret Manager
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")
tavily_ai_api_key = utils.get_tavily_api("TAVILY_API_KEY", aws_region)

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client
bedrock_rt = boto3.client(
    "bedrock-runtime", region_name=aws_region, config=bedrock_config
)

# Create a bedrock client to check available models
bedrock = boto3.client("bedrock", region_name=aws_region, config=bedrock_config)


## ステート マシンとしての LangGraph

システム設計に精通したソリューション アーキテクトにとって、LangGraph は言語モデルのステート マシンと考えることができます。ソフトウェア エンジニアリングのステート マシンが一連の状態と状態間の遷移を定義するのと同様に、LangGraph を使用すると、会話の状態 (ノードで表されます) と状態間の遷移 (エッジで表されます) を定義できます。

**類推**: LangGraph をスマート シティの交通管制システムと考えてください。各交差点 (ノード) は決定ポイントを表し、交差点間の道路 (エッジ) は可能なパスを表します。信号 (条件付きエッジ) は、現在の状況に基づいてどのパスを取るかを決定します。この場合、「トラフィック」は AI エージェント内の情報と決定の流れです。

In [ ]:
import operator
from typing import Annotated, TypedDict

from langchain_aws import ChatBedrockConverse
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langgraph.graph import END, StateGraph

In [ ]:
tool = TavilySearchResults(max_results=4)  # increased number of results
print(type(tool))
print(tool.name)

> Python の型付け注釈に慣れていない場合は、[Python ドキュメント](https://docs.python.org/3/library/typing.html)を参照してください。

## エージェント状態の概念

AgentState クラスは、会話全体を通じてコン​​テキストを維持するために不可欠です。データ サイエンティストにとって、これはリカレント ニューラル ネットワークで状態を維持することに例えることができます。

**類推**: AgentState を洗練されたメモ帳と考えてください。アイデアをブレインストーミング (プロセス クエリ) するときに、重要なポイント (メッセージ) を書き留めます。このメモ帳は単に記録するだけではありません。新しいメモ (メッセージ) が既存のメモとシームレスに統合され、一貫した思考の流れが維持されるという特別な特性があります。
同時に、いつでも時間をさかのぼって一部を書き直すことができます。これを「タイム トラベル」と呼びます。

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> 注: 以下の `take_action` では、LLM が存在しないツール名を返した場合に対応するためにロジックが追加されました。

```python
if not t["name"] in self.tools:  # LLM からの不正なツール名をチェック
    print("\n ....bad tool name....")
    result = "bad tool name, retry"  # 不正な場合は LLM に再試行を指示

```


In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_bedrock)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def call_bedrock(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t["name"] in self.tools:  # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")
        return {"messages": results}

見落とされがちな機能は、Python の関数またはオブジェクトのコードを調べるための `??` です。

`ChatBedrockConverse` クラスの `bind_tools` メソッドを調べてみましょう。

tavily ツールがサポートされるかどうか、また制限があるかどうかわかりますか?

わからない場合は、どのように確認しますか?

In [ ]:
??ChatBedrockConverse.bind_tools

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence).\
Whenever you can, try to call multiple tools at once, to bring down inference time!\
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatBedrockConverse(
    client=bedrock_rt,
    model="anthropic.claude-3-haiku-20240307-v1:0",
    temperature=0,
    max_tokens=None,
)

abot = Agent(model, [tool], system=prompt)

> 追記　プロンプトの翻訳

```
あなたは賢い研究アシスタントです。検索エンジンを使用して情報を検索してください。\
複数の呼び出し（一緒にまたは順番に）を行うことができます。\
可能な限り、複数のツールを一度に呼び出して、推論時間を短縮してください。\
必要な情報が明確になった場合にのみ情報を検索してください。\
フォローアップの質問をする前に情報を検索する必要がある場合は、検索することができます。
```

In [ ]:
# make sure to install pygraphviz if you haven't done so already using 'conda install --channel conda-forge pygraphviz'
from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

In [ ]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
for message in result["messages"]:
    print(f"{message}\n")

In [ ]:
result["messages"][-1].content

In [ ]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result["messages"][-1].content

## 4. 並列ツール呼び出しとシーケンシャル ツール呼び出し

エージェントが並列ツール呼び出しとシーケンシャル ツール呼び出しの両方を実行できる機能は、ソリューション アーキテクトが注意を払うべき強力な機能です。

**詳細**:

- 並列ツール呼び出しは、複数の独立したタスクを同時に実行できるマルチスレッド アプリケーションに似ています。これは、複数の独立した情報を必要とするクエリに効果的です。
- シーケンシャル ツール呼び出しは、1 つの操作の出力が次の操作の入力になるパイプラインに似ています。これは、複数ステップの推論タスクに必要です。

**例え**: 複雑なプロジェクトに取り組んでいる研究チームを想像してください。並列ツール呼び出しは、さまざまな側面を同時に研究するためにさまざまなチーム メンバーを割り当てるようなものです。シーケンシャル ツール呼び出しは、各研究者が前の研究者の調査結果に基づいて構築するリレー レースのようなものです。

シーケンシャル ツール呼び出しと並列ツール呼び出しのどちらがあるかわかりますか。並列の場合、それらは本当に並列で実行されますか。

In [ ]:
# Note, the query was modified to produce more consistent results.
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question."
messages = [HumanMessage(content=query)]

model = ChatBedrockConverse(
    client=bedrock_rt,
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0,
    max_tokens=None,
)
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

In [ ]:
print(result["messages"][-1].content)

# 演習: ツールの並列呼び出しを可能にするには、ツール定義をどのように変更する必要がありますか?

> 注: async を使用すると並列実行を省略できます